# MD5
** Implementation of the RFC-1321 MD5 Algorithm **

*... in Python*

MD5 was proposed in 1991 by Professor Ronald Rivest (The R of the RSA) to replace the MD4 algorithm. The MD5 is a digest algorithm that works on a sequence of 512 bit blocks of data to finally produce a 128 bit hash. This is a one way function as there is no way to recover the original message from the MD5 hash. As of 2012, the MD5 was considered cryptographically broken and unsutable for security applications.

This implementation of MD5 is for demonstrattion purposes to understand and see the MD5 algorithm in action with all intermediate steps.

### Define and Select Test Cases

In [1]:
test_case=[["","d41d8cd98f00b204e9800998ecf8427e"],\
           ["a","0cc175b9c0f1b6a831c399e269772661"],\
           ["abc","900150983cd24fb0d6963f7d28e17f72"],\
           ["message digest","f96b697d7cb7938d525a2f31aaf161d0"],\
           ["abcdefghijklmnopqrstuvwxyz","c3fcd3d76192e4007dfb496cca67e13b"],\
           ["ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789","d174ab98d277d9f5a5611c2c9f419d9f"],\
           ["12345678901234567890123456789012345678901234567890123456789012345678901234567890","57edf4a22be3c955ac49da2e2107b67a"]]
use_test_case = 3
##
message = test_case[use_test_case][0]
ref_hash = test_case[use_test_case][1]

### Step 1 - Append Padding Bits

The messsage to be hashed is padded to have a length equal to 8 bytes {64 bits} less than being a multiple of 64 bytes {512 bits}. The padding step is performed even if the message length is already of desired length. The padding bit string used is `1` followed by `0` - `100...000`

The message length is eventually 56 bytes {448 bits}, 120 bytes {960 bits}, 184 bytes {1472 bits}, 248 bytes {1984 bits} and so on.

In [2]:
message_len = len(message)
message_len_bits = message_len * 8
print("Message Length : " + str(message_len) + " bytes {" + str(message_len_bits) + " bits}")

Message Length : 14 bytes {112 bits}


In [3]:
# Encode string to bytes
message_b = message.encode('utf-8')

In [4]:
# Calculate padding length
padding_len=56-message_len%64
padding_len=64 if (padding_len==0) else padding_len
print("Padding Length : " + str(padding_len) + " bytes {" + str(padding_len * 8) + " bits}")

Padding Length : 42 bytes {336 bits}


In [5]:
# Display Padded Message, length and calculation.
message_mod448 = message_b + b'\x80' + b'\x00' * (padding_len-1)
print("Padded Message :\n"+str(message_mod448))
print("\nlength(paddedMessage)      : "+str(len(message_mod448))+" bytes {"+str(len(message_mod448*8))+" bits}\nlength(paddedMessage) % 64 : "+str(len(message_mod448)%64)+" bytes {"+str((len(message_mod448)%64) * 8)+" bits}" )

Padded Message :
b'message digest\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

length(paddedMessage)      : 56 bytes {448 bits}
length(paddedMessage) % 64 : 56 bytes {448 bits}


### Step 2 - Append Length

The bit length of the original message is appened to this _64 bits short of %512 bit_ message. This bit length is appeneded as an 8 byte {64 bits} little endian integer.

So, a message of length 14 bytes (_try test case # 3_) would have a bit length of 112 bits and the appended 64 bit little endian bit length would be `0x7000000000000000` (as hex) or `b'p\x00\x00\x00\x00\x00\x00\x00'` (as a byte string). If the message length is $> 2^{64}$ bits, only the lower 64 bits are used for padding.

In [6]:
# Append Length
processed_message=message_mod448+(message_len_bits%2**64).to_bytes(8,byteorder='little')
print("LSB64(len(unPaddedMessage)) : "+str((message_len_bits%2**64).to_bytes(8,byteorder='little')))
print("length( paddedMessage | LSB64(len(unPaddedMessage)) ) : "+str(len(processed_message))+" bytes {"+str(len(processed_message)*8)+" bits}")
print("\nPadded Message | LSB64(len(unPaddedMessage)) :\n"+str(processed_message))

LSB64(len(unPaddedMessage)) : b'p\x00\x00\x00\x00\x00\x00\x00'
length( paddedMessage | LSB64(len(unPaddedMessage)) ) : 64 bytes {512 bits}

Padded Message | LSB64(len(unPaddedMessage)) :
b'message digest\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00p\x00\x00\x00\x00\x00\x00\x00'


### Step 3 - Initilize MD Buffer

In [7]:
A = 0x67452301
B = 0xEFCDAB89
C = 0x98BADCFE
D = 0x10325476

### Step 4 - Process Message in 16-Word Blocks

In [8]:
# Auxulary functions that take in 3x 32bit words and return 1x32bit word.

def F(X, Y, Z):
    return ((X&Y) | ((~X) & Z))

def G(X, Y, Z):
    return ((X&Z) | (Y & (~Z)))

def H(X, Y, Z):
    return (X^Y^Z)

def I(X, Y, Z):
    return ( Y^(X|(~Z)) )

In [9]:
# Sine Table
sine_T=[0xD76AA478, 0xE8C7B756, 0x242070DB, 0xC1BDCEEE, 0xF57C0FAF, 0x4787C62A, 0xA8304613, 0xFD469501, 0x698098D8, 0x8B44F7AF, 0xFFFF5BB1, 0x895CD7BE, 0x6B901122, 0xFD987193, 0xA679438E, 0x49B40821, 0xF61E2562, 0xC040B340, 0x265E5A51, 0xE9B6C7AA, 0xD62F105D, 0x02441453, 0xD8A1E681, 0xE7D3FBC8, 0x21E1CDE6, 0xC33707D6, 0xF4D50D87, 0x455A14ED, 0xA9E3E905, 0xFCEFA3F8, 0x676F02D9, 0x8D2A4C8A, 0xFFFA3942, 0x8771F681, 0x6d9d6122, 0xFDE5380C, 0xA4BEEA44, 0x4BDECFA9, 0xF6BB4B60, 0xBEBFBC70, 0x289B7EC6, 0xEAA127FA, 0xD4EF3085, 0x04881D05, 0xD9D4D039, 0xE6DB99E5, 0x1FA27CF8, 0xC4AC5665, 0xF4292244, 0x432AFF97, 0xAB9423A7, 0xFC93A039, 0x655B59C3, 0x8F0CCC92, 0xFFEFF47D, 0x85845DD1, 0x6FA87E4F, 0xFE2CE6E0, 0xA3014314, 0x4E0811A1, 0xF7537E82, 0xBD3AF235, 0x2AD7D2BB, 0xEB86D391]

In [10]:
# Rotate Left
def rotl(x,s):
    return ( (x<<s) | x>>(32-s))

In [11]:
# Shift Table
R1_s=[7,12,17,22]*4
R2_s=[5, 9,14,20]*4
R3_s=[4,11,16,23]*4
R4_s=[6,10,15,21]*4

# K table (to use a sub-string of the message)
R1_k = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
R2_k = [1, 6, 11, 0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12]
R3_k = [5, 8, 11, 14, 1, 4, 7, 10, 13, 0, 3, 6, 9, 12, 15, 2]
R4_k = [0, 7, 14, 5, 12, 3, 10, 1, 8, 15, 6, 13, 4, 11, 2, 9]

In [12]:

def bytereverse(num32):
    rev_byte=0;
    for i in range(0,16):
        #print(hex(num32)+" "+hex(rev_byte))
        rev_byte = rev_byte << 8
        
        low_order_byte = num32 & 0xFF
        rev_byte = rev_byte | low_order_byte
        
        num32 = num32 >> 8
    return rev_byte

In [13]:
## Round Functions
def round1(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(F(b,c,d)))
    FN = F(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    round_output(4,k,s,sine_i,FN)
    return a 

def round2(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(G(b,c,d)))
    FN = G(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    round_output(4,k,s,sine_i,FN)
    return a 

def round3(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(H(b,c,d)))
    FN = H(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    round_output(4,k,s,sine_i,FN)
    return a

def round4(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(I(b,c,d)))
    FN = I(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    round_output(4,k,s,sine_i,FN)
    return a

def round_output(R,k,s,sine_i,FN):
    print("     R"+str(R)+" | K = "+"{:2d}".format(k)+" | s = "+"{:2d}".format(s)+" | sine = "+"{:8x}".format(sine_i)+" | {:9x}".format(FN))
    return None

In [14]:
# Loop though the various 512 bit blocks of a long message.
for i in range(0,len(processed_message),64):
    print("PROCESSING bytes "+str(i)+"..."+str(i+64))
    X  = processed_message[i:i+64]
    print("\nMessage chunk being processed :\n"+str(X)+" \n")
    AA = A
    BB = B
    CC = C
    DD = D
    
    #round 1
    print("*** ROUND 1 ***")
    A = round1(A, B, C, D, X, R1_k[0], sine_T[0], R1_s[0])
    D = round1(D, A, B, C, X, R1_k[1], sine_T[1], R1_s[1])
    C = round1(C, D, A, B, X, R1_k[2], sine_T[2], R1_s[2])
    B = round1(B, C, D, A, X, R1_k[3], sine_T[3], R1_s[3])
    
    A = round1(A, B, C, D, X, R1_k[4], sine_T[4], R1_s[4])
    D = round1(D, A, B, C, X, R1_k[5], sine_T[5], R1_s[5])
    C = round1(C, D, A, B, X, R1_k[6], sine_T[6], R1_s[6])
    B = round1(B, C, D, A, X, R1_k[7], sine_T[7], R1_s[7])
    
    A = round1(A, B, C, D, X, R1_k[8], sine_T[8], R1_s[8])
    D = round1(D, A, B, C, X, R1_k[9], sine_T[9], R1_s[9])
    C = round1(C, D, A, B, X, R1_k[10], sine_T[10], R1_s[10])
    B = round1(B, C, D, A, X, R1_k[11], sine_T[11], R1_s[11])
    
    A = round1(A, B, C, D, X, R1_k[12], sine_T[12], R1_s[12])
    D = round1(D, A, B, C, X, R1_k[13], sine_T[13], R1_s[13])
    C = round1(C, D, A, B, X, R1_k[14], sine_T[14], R1_s[14])
    B = round1(B, C, D, A, X, R1_k[15], sine_T[15], R1_s[15])
    
    print("\n*** ROUND 2 ***")
    A = round2(A, B, C, D, X, R2_k[0], sine_T[16], R2_s[0])
    D = round2(D, A, B, C, X, R2_k[1], sine_T[17], R2_s[1])
    C = round2(C, D, A, B, X, R2_k[2], sine_T[18], R2_s[2])
    B = round2(B, C, D, A, X, R2_k[3], sine_T[19], R2_s[3])
    
    A = round2(A, B, C, D, X, R2_k[4], sine_T[20], R2_s[4])
    D = round2(D, A, B, C, X, R2_k[5], sine_T[21], R2_s[5])
    C = round2(C, D, A, B, X, R2_k[6], sine_T[22], R2_s[6])
    B = round2(B, C, D, A, X, R2_k[7], sine_T[23], R2_s[7])
    
    A = round2(A, B, C, D, X, R2_k[8], sine_T[24], R2_s[8])
    D = round2(D, A, B, C, X, R2_k[9], sine_T[25], R2_s[9])
    C = round2(C, D, A, B, X, R2_k[10], sine_T[26], R2_s[10])
    B = round2(B, C, D, A, X, R2_k[11], sine_T[27], R2_s[11])
    
    A = round2(A, B, C, D, X, R2_k[12], sine_T[28], R2_s[12])
    D = round2(D, A, B, C, X, R2_k[13], sine_T[29], R2_s[13])
    C = round2(C, D, A, B, X, R2_k[14], sine_T[30], R2_s[14])
    B = round2(B, C, D, A, X, R2_k[15], sine_T[31], R2_s[15])
    
    print("\n*** ROUND 3 ***")
    A = round3(A, B, C, D, X, R3_k[0], sine_T[32], R3_s[0])
    D = round3(D, A, B, C, X, R3_k[1], sine_T[33], R3_s[1])
    C = round3(C, D, A, B, X, R3_k[2], sine_T[34], R3_s[2])
    B = round3(B, C, D, A, X, R3_k[3], sine_T[35], R3_s[3])
    
    A = round3(A, B, C, D, X, R3_k[4], sine_T[36], R3_s[4])
    D = round3(D, A, B, C, X, R3_k[5], sine_T[37], R3_s[5])
    C = round3(C, D, A, B, X, R3_k[6], sine_T[38], R3_s[6])
    B = round3(B, C, D, A, X, R3_k[7], sine_T[39], R3_s[7])
    
    A = round3(A, B, C, D, X, R3_k[8], sine_T[40], R3_s[8])
    D = round3(D, A, B, C, X, R3_k[9], sine_T[41], R3_s[9])
    C = round3(C, D, A, B, X, R3_k[10], sine_T[42], R3_s[10])
    B = round3(B, C, D, A, X, R3_k[11], sine_T[43], R3_s[11])
    
    A = round3(A, B, C, D, X, R3_k[12], sine_T[44], R3_s[12])
    D = round3(D, A, B, C, X, R3_k[13], sine_T[45], R3_s[13])
    C = round3(C, D, A, B, X, R3_k[14], sine_T[46], R3_s[14])
    B = round3(B, C, D, A, X, R3_k[15], sine_T[47], R3_s[15])
    
    print("\n*** ROUND 4 ***")
    A = round4(A, B, C, D, X, R4_k[0], sine_T[48], R4_s[0])    
    D = round4(D, A, B, C, X, R4_k[1], sine_T[49], R4_s[1])    
    C = round4(C, D, A, B, X, R4_k[2], sine_T[50], R4_s[2])    
    B = round4(B, C, D, A, X, R4_k[3], sine_T[51], R4_s[3])    
    
    A = round4(A, B, C, D, X, R4_k[4], sine_T[52], R4_s[4])    
    D = round4(D, A, B, C, X, R4_k[5], sine_T[53], R4_s[5])    
    C = round4(C, D, A, B, X, R4_k[6], sine_T[54], R4_s[6])    
    B = round4(B, C, D, A, X, R4_k[7], sine_T[55], R4_s[7])    
    
    A = round4(A, B, C, D, X, R4_k[8], sine_T[56], R4_s[8])    
    D = round4(D, A, B, C, X, R4_k[9], sine_T[57], R4_s[9])    
    C = round4(C, D, A, B, X, R4_k[10], sine_T[58], R4_s[10])
    B = round4(B, C, D, A, X, R4_k[11], sine_T[59], R4_s[11])    
    
    A = round4(A, B, C, D, X, R4_k[12], sine_T[60], R4_s[12])
    D = round4(D, A, B, C, X, R4_k[13], sine_T[61], R4_s[13])
    C = round4(C, D, A, B, X, R4_k[14], sine_T[62], R4_s[14])
    B = round4(B, C, D, A, X, R4_k[15], sine_T[63], R4_s[15])
    
    # Update the MD buffer after processing each 512 bit block.
    A = (A + AA) & 0xFFFFFFFF
    B = (B + BB) & 0xFFFFFFFF
    C = (C + CC) & 0xFFFFFFFF
    D = (D + DD) & 0xFFFFFFFF
    
    # Display Updated MD Buffers
    print("\n*** MD Buffers after processing chunk ***\n[D C B A] = "+"[{:8x} {:8x} {:8x} {:8x}]".format(D,C,B,A)+"\n\n")

PROCESSING bytes 0...64

Message chunk being processed :
b'message digest\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00p\x00\x00\x00\x00\x00\x00\x00' 

*** ROUND 1 ***
     R4 | K =  0 | s =  7 | sine = d76aa478 |  98badcfe
     R4 | K =  1 | s = 12 | sine = e8c7b756 |  cee8c9d8
     R4 | K =  2 | s = 17 | sine = 242070db |  4fcf9fab
     R4 | K =  3 | s = 22 | sine = c1bdceee |  cea2fdba
     R4 | K =  4 | s =  7 | sine = f57c0faf |  d232f032
     R4 | K =  5 | s = 12 | sine = 4787c62a |  b8112052
     R4 | K =  6 | s = 17 | sine = a8304613 |  b910cd94
     R4 | K =  7 | s = 22 | sine = fd469501 |  faebd384
     R4 | K =  8 | s =  7 | sine = 698098d8 |  b889c502
     R4 | K =  9 | s = 12 | sine = 8b44f7af |  c40734e9
     R4 | K = 10 | s = 17 | sine = ffff5bb1 |  dc86d4dd
     R4 | K = 11 | s = 22 | sine = 895cd7be |  bbfed5d8
     R4 | K = 12 | s =  7 | sine = 6b9

In [15]:
# Compute output hash from the MD buffers.
output_int = D<<96 | C <<64 | B << 32 | A

# The MD5 hash starts with the lowest order byte of A ... highest order byte of D
print("OUTPUT      : "+hex(bytereverse(output_int)))
print("REF. Hash   : 0x"+test_case[use_test_case][1])

OUTPUT      : 0xf96b697d7cb7938d525a2f31aaf161d0
REF. Hash   : 0xf96b697d7cb7938d525a2f31aaf161d0


## Compare with Python's `hashlib`

In [16]:
import hashlib

In [17]:
H = hashlib.new('md5')
H.update(message_b)
md5hash=H.hexdigest()
print("Hashlib MD5 : 0x"+md5hash)

Hashlib MD5 : 0xf96b697d7cb7938d525a2f31aaf161d0


### References

1. [RFC-1321](https://tools.ietf.org/html/rfc1321)
2. [Wikipedia](https://en.wikipedia.org/wiki/MD5)
3. [Rosetta Code](https://rosettacode.org/wiki/MD5/Implementation#Python)